# The dipole model, updated

In [1]:
import tensorflow as tf
from pinn.optimizers import get
from glob import glob
from pinn.io import load_qm9
from docs.notebooks.network_fns import get_traintest_sets, preprocess_traintest_sets
import time
from pinn.layers import PolynomialBasis, GaussianBasis, ANNOutput
from pinn.networks.pinet import OutLayer, GCBlock, ResUpdate, PreprocessLayer

Init Plugin
Init Graph Optimizer
Init Kernel


In [ ]:
# Run this cell to train on CPU
physical_devices = tf.config.list_physical_devices()
tf.config.set_visible_devices(physical_devices[0], 'CPU')
tf.config.set_visible_devices([], 'GPU')

In [2]:
filelist = glob('/Users/miguelnavaharris/Project/QM9/*.xyz')
dataset = load_qm9(filelist, label_map={'d_data': 'mu'}, splits={'train':8, 'test':2}) 

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-08-01 02:05:59.482985: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-01 02:05:59.483089: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
params={
    'optimizer': {
        'class_name': 'Adam',
        'config': {
            'learning_rate': {
                'class_name': 'ExponentialDecay',
                'config': {
                    'initial_learning_rate': 0.0003,
                    'decay_steps': 10000, 
                    'decay_rate': 0.994}}, 
                    'clipnorm': 0.01}}
}

In [4]:
class PiNet(tf.keras.Model):
    """Keras model for the PiNet neural network

    Args:
        tensors: input data (nested tensor from dataset).
        atom_types (list): elements for the one-hot embedding.
        pp_nodes (list): number of nodes for pp layer.
        pi_nodes (list): number of nodes for pi layer.
        ii_nodes (list): number of nodes for ii layer.
        en_nodes (list): number of nodes for en layer.
        depth (int): number of interaction blocks.
        rc (float): cutoff radius.
        basis_type (string): type of basis function to use,
            can be "polynomial" or "gaussian".
        n_basis (int): number of basis functions to use.
        gamma (float or array): width of gaussian function for gaussian basis.
        center (float or array): center of gaussian function for gaussian basis.
        cutoff_type (string): cutoff function to use with the basis.
        act (string): activation function to use.
        preprocess (bool): whether to return the preprocessed tensor.
    """
    def __init__(self, atom_types=[1, 6, 7, 8, 9],  rc=4.0, cutoff_type='f1',
                 basis_type='polynomial', n_basis=4, gamma=3.0, center=None,
                 pp_nodes=[16, 16], pi_nodes=[16, 16], ii_nodes=[16, 16],
                 out_nodes=[16, 16], out_units=1, out_pool=False,
                 act='tanh', depth=4):

        super(PiNet, self).__init__()

        self.depth = depth
        self.preprocess = PreprocessLayer(atom_types, rc)
        self.activation = act

        if basis_type == 'polynomial':
            self.basis_fn = PolynomialBasis(cutoff_type, rc, n_basis)
        elif basis_type == 'gaussian':
            self.basis_fn = GaussianBasis(cutoff_type, rc, n_basis, gamma, center)

        self.res_update = [ResUpdate() for i in range(depth)]
        self.gc_blocks = [GCBlock([], pi_nodes, ii_nodes, activation=act)]
        self.gc_blocks += [GCBlock(pp_nodes, pi_nodes, ii_nodes, activation=act)
                           for i in range(depth-1)]
        self.out_layers = [OutLayer(out_nodes, out_units) for i in range(depth)]
        self.ann_output =  ANNOutput(out_pool)
    
    def call(self, tensors):
        tensors = self.preprocess(tensors)
        basis = self.basis_fn(tensors['dist'])[:, None, :]

        output = 0.0
        for i in range(self.depth):
            prop = self.gc_blocks[i]([tensors['ind_2'], tensors['prop'], basis])
            output = self.out_layers[i]([tensors['ind_1'], prop, output])
            tensors['prop'] = self.res_update[i]([tensors['prop'], prop])

        output = self.ann_output([tensors['ind_1'], output])
        return output

In [5]:
def train_and_evaluate_network(network=None, params=None, train_set=None, test_set=None, batch_size=256, epochs=1):


    # Instantiate an optimizer
    optimizer = get(params['optimizer'])
    # Define metrics
    D_MAE = tf.keras.metrics.MeanAbsoluteError()
    D_RMSE = tf.keras.metrics.RootMeanSquaredError()
    Q_MAE = tf.keras.metrics.MeanAbsoluteError()
    Q_RMSE = tf.keras.metrics.RootMeanSquaredError()




    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        start_time_epoch = time.time()
        hund_step_times = []


        # Iterate over the batches of the dataset.
        for step, batch in enumerate(train_set):

            train_losses = []

            with tf.GradientTape() as tape:

                # start_time = time.time()

                pred = network(batch, training=True)  # Logits for this minibatch
                pred = tf.expand_dims(pred, axis=1)
                ind = batch['ind_1']
                nbatch = tf.reduce_max(ind)+1
                charge = tf.math.unsorted_segment_sum(pred, ind[:, 0], nbatch)
                dipole = pred * batch['coord']
                dipole = tf.math.unsorted_segment_sum(dipole, ind[:, 0], nbatch)
                dipole = tf.sqrt(tf.reduce_sum(dipole**2, axis=1)+1e-6)

                d_data = batch['d_data']
                q_data = tf.zeros_like(charge)

                d_error = d_data - dipole
                q_error = q_data - charge
                d_loss = tf.reduce_mean(d_error**2)
                train_losses.append(d_loss)
                q_loss = tf.reduce_mean(q_error**2)
                train_losses.append(q_loss)


                # Compute the loss value for this minibatch.
                train_loss_value = tf.reduce_sum(train_losses)


            grads = tape.gradient(train_loss_value, network.trainable_weights)


            optimizer.apply_gradients(zip(grads, network.trainable_weights))

            # end_time = time.time() - start_time
            # print('End time', end_time)

            # Log every 20 batches.
            if step == 0:
                print(f"Initial loss (for one batch): {float(train_loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")



            elif step % 100 == 0:
                print(f"Training loss (for one batch) at step {step}: {float(train_loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")
                hund_step_times += [(time.time() - start_time_epoch)]
                print(f'Training time for 100 batches: {((hund_step_times[-1] - hund_step_times[-2]) if len(hund_step_times) > 1 else hund_step_times[-1])} s')

      
        print(f'Training time for epoch {epoch + 1}: {(time.time() - start_time_epoch)} s')        

        

        # Run a validation loop at the end of each epoch
        print(f'Starting validation for epoch {(epoch + 1)}')

        for step, batch in enumerate(test_set):

            val_losses = []

            val_pred = network(batch, training=False) # Logits for this minibatch
            val_pred = tf.expand_dims(val_pred, axis=1)
            ind = batch['ind_1']
            nbatch = tf.reduce_max(ind)+1
            charge = tf.math.unsorted_segment_sum(val_pred, ind[:, 0], nbatch)
            dipole = val_pred * batch['coord']
            dipole = tf.math.unsorted_segment_sum(dipole, ind[:, 0], nbatch)
            dipole = tf.sqrt(tf.reduce_sum(dipole**2, axis=1)+1e-6)

            d_data = batch['d_data']
            q_data = tf.zeros_like(charge)
            
            d_error = d_data - dipole
            q_error = q_data - charge
            d_loss = tf.reduce_mean(d_error**2)
            val_losses.append(d_loss)
            q_loss = tf.reduce_mean(q_error**2)
            val_losses.append(q_loss)

            # Compute the loss value for this minibatch.
            val_loss_value = tf.reduce_sum(val_losses)

            # Update metrics
            D_MAE.update_state(d_data, dipole)
            D_RMSE.update_state(d_data, dipole)
            Q_MAE.update_state(q_data, charge)
            Q_RMSE.update_state(q_data, charge)


            # Log every 20 batches.
            if step == 0:
                print(f"Initial loss (for one batch): {float(val_loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")


            elif step % 100 == 0:
                print(f"Validation loss (for one batch) at step {step}: {float(val_loss_value)}")
                print(f"Seen so far: {((step + 1) * batch_size)} molecules")
                hund_step_times += [(time.time() - start_time_epoch)]
                print(f'Validation time for 100 batches: {((hund_step_times[-1] - hund_step_times[-2]) if len(hund_step_times) > 1 else hund_step_times[-1])} s')
            
        # Display and reset validation metric results
        print(f"D_MAE: {float(D_MAE.result())}")
        print(f"D_RMSE: {float(D_RMSE.result())}")
        print(f"Q_MAE: {float(Q_MAE.result())}")
        print(f"Q_RMSE: {float(Q_RMSE.result())}")
        D_MAE.reset_states()
        D_RMSE.reset_states()
        Q_MAE.reset_states()
        Q_RMSE.reset_states()


        print(f"Time taken for epoch {epoch + 1}: {(time.time() - start_time_epoch)} s")
        

In [6]:
train_set, test_set, batch_size = get_traintest_sets(dataset=dataset, buffer_size=1000, batch_size=256)
network = PiNet()
preprocess_traintest_sets(train_set, test_set, network=network)


2022-08-01 02:06:03.657411: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-01 02:06:03.657634: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [7]:
train_and_evaluate_network(network, params=params, train_set=train_set, test_set=test_set, batch_size=batch_size, epochs=1)


Start of epoch 0
Initial loss (for one batch): 1779.81689453125
Seen so far: 256 molecules
Training loss (for one batch) at step 100: 32.78116226196289
Seen so far: 25856 molecules
Training time for 100 batches: 55.77427625656128 s
Training loss (for one batch) at step 200: 6.726493835449219
Seen so far: 51456 molecules
Training time for 100 batches: 71.51356387138367 s
Training loss (for one batch) at step 300: 2.994511604309082
Seen so far: 77056 molecules
Training time for 100 batches: 79.5966010093689 s
Training loss (for one batch) at step 400: 1.2767127752304077
Seen so far: 102656 molecules
Training time for 100 batches: 91.73267698287964 s
Training time for epoch 1: 316.51854705810547 s
Starting validation for epoch 1
Initial loss (for one batch): 170.52392578125
Seen so far: 256 molecules
Validation loss (for one batch) at step 100: 165.39002990722656
Seen so far: 25856 molecules
Validation time for 100 batches: 65.2418122291565 s
D_MAE: 9.314730644226074
D_RMSE: 11.271841049